In [6]:
import torch
from torch.utils import data

In [3]:
a = torch.tensor([0, 1, 2]).reshape(-1, 1)
a, a.shape

(tensor([[0],
         [1],
         [2]]),
 torch.Size([3, 1]))

In [4]:
b = torch.tensor([0, 1]).reshape(1, -1)
b, b.shape

(tensor([[0, 1]]), torch.Size([1, 2]))

In [5]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

In [10]:
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1,  size= (num_examples, len(w)))
    y = torch.matmul(X,w) 
    y += b
    y += torch.normal(0, 0.01, size=(y.shape))
    return X, y.reshape(-1,1)

In [11]:
#help(torch.normal)

In [12]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
N = 1000
features, labels = synthetic_data(true_w, true_b, N)

features.shape, labels.shape

(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [14]:
help(data.DataLoader)

Help on class DataLoader in module torch.utils.data.dataloader:

class DataLoader(typing.Generic)
 |  DataLoader(dataset: torch.utils.data.dataset.Dataset[+T_co], batch_size: Optional[int] = 1, shuffle: Optional[bool] = None, sampler: Union[torch.utils.data.sampler.Sampler, Iterable, NoneType] = None, batch_sampler: Union[torch.utils.data.sampler.Sampler[Sequence], Iterable[Sequence], NoneType] = None, num_workers: int = 0, collate_fn: Optional[Callable[[List[~T]], Any]] = None, pin_memory: bool = False, drop_last: bool = False, timeout: float = 0, worker_init_fn: Optional[Callable[[int], NoneType]] = None, multiprocessing_context=None, generator=None, *, prefetch_factor: Optional[int] = None, persistent_workers: bool = False, pin_memory_device: str = '')
 |  
 |  Data loader. Combines a dataset and a sampler, and provides an iterable over
 |  the given dataset.
 |  
 |  The :class:`~torch.utils.data.DataLoader` supports both map-style and
 |  iterable-style datasets with single- or mu

In [15]:
help(data.TensorDataset)

Help on class TensorDataset in module torch.utils.data.dataset:

class TensorDataset(Dataset)
 |  TensorDataset(*tensors: torch.Tensor) -> None
 |  
 |  Dataset wrapping tensors.
 |  
 |  Each sample will be retrieved by indexing tensors along the first dimension.
 |  
 |  Args:
 |      *tensors (Tensor): tensors that have the same size of the first dimension.
 |  
 |  Method resolution order:
 |      TensorDataset
 |      Dataset
 |      typing.Generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, index)
 |  
 |  __init__(self, *tensors: torch.Tensor) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __len__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __annotations__ = {'tensors': typing.Tuple[torch.Tensor, ...]}
 |  
 |  __orig_bases__ = (torch.utils.data.dataset.Dataset[typing.Tuple[torch....
 |  
 |  __parameters__ 

In [17]:
b = (1, 2)
*b

SyntaxError: can't use starred expression here (3999097260.py, line 2)

In [ ]:
list(zip(features, labels))

tensor([-1.2807,  0.1441])

In [24]:
def load_array(data_arrays, batch_size, is_train=True):
    """Construct a Pytorch data iterator"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features,labels), batch_size)

In [29]:
 xs = [10, 20, 30] ## this is an iterable
 it = iter(xs)    ## this is an iterator object

In [37]:
next(iter(data_iter))

[tensor([[-0.6448, -0.4955],
         [ 0.1100, -0.9053],
         [ 0.4761,  0.7436],
         [ 1.1318, -0.2649],
         [ 0.3704, -1.2170],
         [-2.7935,  0.7753],
         [ 0.3697,  1.1704],
         [ 1.3909,  0.2014],
         [-0.2749, -1.4179],
         [ 1.2132, -0.3327]]),
 tensor([[ 4.5873],
         [ 7.5036],
         [ 2.6378],
         [ 7.3694],
         [ 9.0706],
         [-4.0241],
         [ 0.9793],
         [ 6.2941],
         [ 8.4685],
         [ 7.7547]])]

## 2. Model

In [40]:
from torch import nn

In [41]:
net = nn.Sequential(nn.Linear(2, 1))

In [47]:
net(features).shape, list(net.parameters())

(torch.Size([1000, 1]),
 [Parameter containing:
  tensor([[-0.6666, -0.1235]], requires_grad=True),
  Parameter containing:
  tensor([0.4765], requires_grad=True)])

In [48]:
net.state_dict()

OrderedDict([('0.weight', tensor([[-0.6666, -0.1235]])),
             ('0.bias', tensor([0.4765]))])

In [52]:
net[0], list(net[0].parameters())

(Linear(in_features=2, out_features=1, bias=True),
 [Parameter containing:
  tensor([[-0.6666, -0.1235]], requires_grad=True),
  Parameter containing:
  tensor([0.4765], requires_grad=True)])

In [54]:
list(net.parameters()) == list(net[0].parameters())

True

In [60]:
net[0].state_dict()['weight'], net[0].weight, net[0].bias

(tensor([[-0.6666, -0.1235]]),
 Parameter containing:
 tensor([[-0.6666, -0.1235]], requires_grad=True),
 Parameter containing:
 tensor([0.4765], requires_grad=True))

In [61]:
## initialise w and b
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [62]:
net[0].state_dict()['weight'], net[0].weight, net[0].bias

(tensor([[0.0033, 0.0126]]),
 Parameter containing:
 tensor([[0.0033, 0.0126]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True))

# 3. Optimizer

In [63]:
loss = nn.MSELoss()

In [65]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

# 4. training

In [68]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f"epoch {epoch + 1}, loss {l:f}")

epoch 1, loss 0.000106
epoch 2, loss 0.000104
epoch 3, loss 0.000104


In [69]:
net.state_dict()

OrderedDict([('0.weight', tensor([[ 1.9990, -3.3995]])),
             ('0.bias', tensor([4.2000]))])

OOP design - Pytorch

In [72]:
import torch
from torch import nn
device = "cpu"

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [73]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [74]:
X = torch.randn(1, 28, 28, device=device)
logits = model(X)
print(logits)

tensor([[ 0.0050, -0.0153,  0.0683,  0.0714, -0.0620, -0.0387, -0.0522,  0.0187,
          0.0862,  0.0225]], grad_fn=<AddmmBackward0>)


In [78]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab, pred_probab.sum()

(tensor([[0.0993, 0.0973, 0.1058, 0.1062, 0.0929, 0.0951, 0.0938, 0.1007, 0.1077,
          0.1011]], grad_fn=<SoftmaxBackward0>),
 tensor(1., grad_fn=<SumBackward0>))

In [80]:
for name, param in model.named_parameters():
    print(f"Layer: {name}, Size:{param.shape}")


Layer: linear_relu_stack.0.weight, Size:torch.Size([512, 784])
Layer: linear_relu_stack.0.bias, Size:torch.Size([512])
Layer: linear_relu_stack.2.weight, Size:torch.Size([512, 512])
Layer: linear_relu_stack.2.bias, Size:torch.Size([512])
Layer: linear_relu_stack.4.weight, Size:torch.Size([10, 512])
Layer: linear_relu_stack.4.bias, Size:torch.Size([10])
